In [26]:
import pandas as pd
cuisines_df = pd.read_csv("data/cleaned_cuisines.csv")
cuisines_df.head()

,Unnamed: 0,cuisine,almond,angelica,anise,anise_seed,apple,apple_brandy,apricot,armagnac,...,whiskey,white_bread,white_wine,whole_grain_wheat_flour,wine,wood,yam,yeast,yogurt,zucchini
0,0,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,indian,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [114]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score,precision_score,confusion_matrix,classification_report, precision_recall_curve
from sklearn.svm import SVC
import numpy as np

In [116]:
# Turn cuisines into label dataframe (y)
cuisines_label_df = cuisines_df['cuisine']
cuisines_label_df.head()

0    indian
1    indian
2    indian
3    indian
4    indian
Name: cuisine, dtype: object

In [118]:
# Drop "Unnamed: 0" and "cuisine" column. Rest are trainable features (X)
cuisines_feature_df = cuisines_df.drop(['Unnamed: 0', 'cuisine'], axis=1)
cuisines_feature_df.head()

,almond,angelica,anise,anise_seed,apple,apple_brandy,apricot,armagnac,artemisia,artichoke,...,whiskey,white_bread,white_wine,whole_grain_wheat_flour,wine,wood,yam,yeast,yogurt,zucchini
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


# Choose a Classifier

Looking for multiclass classification. 

Options:
- Multiclass Logistic Regression = Fast trainng times, linear model
- Muliticlass Neural Network = Accuracy, long training times
- Multiclass Decision Forest = Accuracy, fast training times
- One-vs-All Multiclass = Depends on two-class classifier
- Multiclass Boosted Decision tree = Non-parametric, fast training times and scalable


Reasoning:
- Neural networks too heavy
- No two-class classifier
- No need for ranking
- -> Try decision tree or multiclass logistic regression

In [122]:
# Split into train and test groups
X_train, X_test, y_train, y_test = train_test_split(cuisines_feature_df, cuisines_label_df, test_size=0.3)

In [124]:
print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_train.shape}')
print(f'y_train shape: {X_train.shape}')
print(f'y_test shape: {X_train.shape}')

X_train shape: (2796, 378)
X_test shape: (2796, 378)
y_train shape: (2796, 378)
y_test shape: (2796, 378)


In [126]:
X_train.head()

,almond,angelica,anise,anise_seed,apple,apple_brandy,apricot,armagnac,artemisia,artichoke,...,whiskey,white_bread,white_wine,whole_grain_wheat_flour,wine,wood,yam,yeast,yogurt,zucchini
1222,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1427,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2673,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3268,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3132,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [128]:
y_train.head()

1222      korean
1427      korean
2673     chinese
3268    japanese
3132    japanese
Name: cuisine, dtype: object

In [218]:
# Apply logistic regression 
# (with multiclass one-vs-rest scheme and liblinear solver)
lr = OneVsRestClassifier(LogisticRegression(solver='liblinear'))
model = lr.fit(X_train, np.ravel(y_train))

accuracy = model.score(X_test, y_test)
print ("Accuracy is {}".format(accuracy))

Accuracy is 0.780650542118432


In [258]:
# Predict one random row of data

random_idx = np.random.randint(0, len(X_test))
ingredients = X_test.iloc[random_idx][X_test.iloc[random_idx] != 0].index.tolist()
print(f'Ingredients for Row {random_idx}: {ingredients}')

pred = model.predict(X_test.iloc[[random_idx]])[0]
print(f'\nPredicted cuisine: {pred}')

print(f'True cuisine: {y_test.iloc[random_idx]}')

Ingredients for Row 64: ['clam', 'roasted_sesame_seed', 'scallion', 'vinegar']

Predicted cuisine: korean
True cuisine: korean


In [260]:
# Check accuracy of prediction with probabilities for each cuisine

test = X_test.iloc[[random_idx]]
proba = model.predict_proba(test)
classes = model.classes_
resultdf = pd.DataFrame(data=proba, columns=classes)

topPrediction = resultdf.T.sort_values(by=[0], ascending = [False])
topPrediction.head()

,0
korean,0.919856
japanese,0.041443
thai,0.018706
indian,0.011261
chinese,0.008733


In [264]:
# Print classification report for more detail
y_pred = model.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

     chinese       0.73      0.66      0.69       257
      indian       0.94      0.94      0.94       232
    japanese       0.70      0.73      0.72       249
      korean       0.80      0.75      0.77       228
        thai       0.75      0.85      0.79       233

    accuracy                           0.78      1199
   macro avg       0.78      0.78      0.78      1199
weighted avg       0.78      0.78      0.78      1199



# Try Other Classifers

In [271]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score,precision_score,confusion_matrix,classification_report, precision_recall_curve

In [306]:
C = 10
# Create different classifiers.
classifiers = {
    'Linear SVC': SVC(kernel='linear', C=C, probability=True,random_state=0),
    'KNN classifier': KNeighborsClassifier(C),
    'SVC': SVC(),
    'RFST': RandomForestClassifier(n_estimators=100),
    'ADA': AdaBoostClassifier(n_estimators=100, algorithm='SAMME')
}

In [313]:
# Train model with Linear SVC, K-Neighbors, SVC, Random Forest, and AdaBoost
n_classifiers = len(classifiers)

for index, (name, classifier) in enumerate(classifiers.items()):
    classifier.fit(X_train, np.ravel(y_train))

    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy (train) for %s: %0.1f%% " % (name, accuracy * 100))
    print(classification_report(y_test,y_pred))

Accuracy (train) for Linear SVC: 77.9% 
              precision    recall  f1-score   support

     chinese       0.70      0.71      0.70       257
      indian       0.91      0.88      0.90       232
    japanese       0.78      0.71      0.75       249
      korean       0.79      0.73      0.76       228
        thai       0.74      0.86      0.80       233

    accuracy                           0.78      1199
   macro avg       0.78      0.78      0.78      1199
weighted avg       0.78      0.78      0.78      1199

Accuracy (train) for KNN classifier: 71.1% 
              precision    recall  f1-score   support

     chinese       0.63      0.74      0.68       257
      indian       0.84      0.81      0.83       232
    japanese       0.60      0.80      0.69       249
      korean       0.91      0.52      0.66       228
        thai       0.74      0.66      0.70       233

    accuracy                           0.71      1199
   macro avg       0.74      0.71      0.71    

### Random forest seems to do the best, followed by SVC